# 目录
- [面向初学者的快速入门](#面向初学者的快速入门)
- [面向专家的快速入门](#面向专家的快速入门)
- [tensorflow2改变了什么](#tensorflow2改变了什么)
- [推荐使用技巧](#推荐使用技巧)

## 面向初学者的快速入门

源自：https://www.tensorflow.org/tutorials/quickstart/beginner

[回到目录](#目录)

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 安装 TensorFlow

import tensorflow as tf
print(tf.__version__)         # 2.0.0-alpha0
print(tf.keras.__version__)   # 2.2.4-tf

# 载入并准备好 MNIST 数据集。将样本从整数转换为浮点数
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 将模型的各层堆叠起来，以搭建 tf.keras.Sequential 模型。为训练选择优化器和损失函数
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 训练并验证
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

2.0.0-alpha0
2.2.4-tf
Epoch 1/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.3001 - accuracy: 0.9122
Epoch 2/5
60000/60000 [==============================] - 4s 69us/sample - loss: 0.1472 - accuracy: 0.9564
Epoch 3/5
60000/60000 [==============================] - 4s 71us/sample - loss: 0.1090 - accuracy: 0.9675
Epoch 4/5
60000/60000 [==============================] - 4s 69us/sample - loss: 0.0883 - accuracy: 0.9726
Epoch 5/5
60000/60000 [==============================] - 4s 68us/sample - loss: 0.0754 - accuracy: 0.9764
10000/10000 - 0s - loss: 0.0718 - accuracy: 0.9774


[0.07179369834843091, 0.9774]

## 面向专家的快速入门

源自：https://www.tensorflow.org/tutorials/quickstart/advanced

[回到目录](#目录)

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

# 加载并准备 MNIST 数据集
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension:(x,y,z) => (x,y,z,1)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# 使用 tf.data 来将数据集切分为 batch 以及混淆数据集
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 使用 Keras 模型子类化（model subclassing） API 构建 tf.keras 模型
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()


# 为训练选择优化器与损失函数
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 选择衡量指标来度量模型的损失值（loss）和准确率（accuracy）。这些指标在 epoch 上累积值，然后打印出整体结果
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [3]:
# 使用 tf.GradientTape 来训练模型

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


# 测试模型
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

Epoch 1, Loss: 0.13203899562358856, Accuracy: 96.05833435058594, Test Loss: 0.06168621405959129, Test Accuracy: 97.90999603271484
Epoch 2, Loss: 0.085730642080307, Accuracy: 97.43499755859375, Test Loss: 0.05973690375685692, Test Accuracy: 97.95500183105469
Epoch 3, Loss: 0.06337665021419525, Accuracy: 98.0999984741211, Test Loss: 0.05973971262574196, Test Accuracy: 98.05333709716797
Epoch 4, Loss: 0.05047867074608803, Accuracy: 98.4808349609375, Test Loss: 0.060349710285663605, Test Accuracy: 98.12249755859375
Epoch 5, Loss: 0.04229719936847687, Accuracy: 98.72100067138672, Test Loss: 0.06144608557224274, Test Accuracy: 98.15800476074219


### 问题：为什么用了```tf.GradientTape()```？ 
[解答位于：推荐使用技巧 — eager执行]()

## tensorflow2改变了什么

源自：https://www.tensorflow.org/guide/effective_tf2

[回到目录](#目录)

- **API Cleanup**

重整了很多混乱的模块

- **Eager execution**

TensorFlow虽是深度学习领域最广泛使用的框架，但是对比PyTorch这一动态图框架，采用静态图（Graph模式）的TensorFlow确实是难用。

好在最近TensorFlow支持了eager模式，对标PyTorch的动态执行机制。

```python
# 在tensorflow2中默认使用Eager Execution
tf.executing_eagerly()
```


- **No more globals**

变量的定义更加便捷，不像 tensorflow 1.X 那么复杂

- **Functions, not sessions**
```python
# TensorFlow 1.X
outputs = session.run(f(placeholder), feed_dict={placeholder: input})
# TensorFlow 2.0
outputs = f(input)
```

**AutoGraph** converts a subset of Python constructs into their TensorFlow equivalents:
```python
for/while -> tf.while_loop (break and continue are supported)
if -> tf.cond
for _ in dataset -> dataset.reduce
```


## 推荐使用技巧

- [eager执行](#1.eager执行)
- [AutoGraph](#2.AutoGraph)
- [tf.function性能优化](#3.tf.function性能优化)
- [tf.keras模型构建](#4.tf.keras模型构建)
- [模型训练](#5.模型训练)

源自：

https://www.tensorflow.org/guide/effective_tf2

https://mp.weixin.qq.com/s/1I9QR-xoum0IfofsRM1c5A

https://mp.weixin.qq.com/s/5ygXEMr7DAEPN_G1V_Qkug


[回到目录](#目录)

### 1.eager执行

TensorFlow的Eager执行时一种命令式编程（imperative programming），这和原生Python是一致的，当你执行某个操作时是立即返回结果的。

而TensorFlow一直是采用Graph模式，即先构建一个计算图，然后需要开启Session，喂进实际的数据才真正执行得到结果。显然，**eager执行更简洁，我们可以更容易debug自己的代码，这也是为什么PyTorch更简单好用的原因**。

In [21]:
x = tf.ones((2, 2), dtype=tf.dtypes.float32)
y = tf.constant([[1, 2],
                 [3, 4]], dtype=tf.dtypes.float32)
z = tf.matmul(x, y)
print(z)
print("------------")
print(z.numpy())    #获得Tensor所对应的numpy数组

tf.Tensor(
[[4. 6.]
 [4. 6.]], shape=(2, 2), dtype=float32)
------------
[[4. 6.]
 [4. 6.]]


#### 1.1 可以直接看到结果

上面例子可以看到在eager执行下，每个操作后的返回值是tf.Tensor，其包含具体值，不再像Graph模式下那样只是一个计算图节点的符号句柄。

由于可以立即看到结果，这非常有助于程序debug。更进一步地，```调用tf.Tensor.numpy()方法可以获得Tensor所对应的numpy数组```。

#### 1.2 可以使用Python原生功能

```python
random_value = tf.random.uniform([], 0, 1)
x = tf.reshape(tf.range(0, 4), [2, 2])
print(random_value)
if random_value.numpy() > 0.5:
    y = tf.matmul(x, x)
else:
    y = tf.add(x, x)
    
# tf.Tensor(0.6259608, shape=(), dtype=float32)
```

这种动态控制流主要得益于eager执行得到Tensor可以取出numpy值，这避免了使用Graph模式下的tf.cond和tf.while等算子。

#### 1.3 tf.GradientTape()是什么？

**一个重要的问题，在egaer模式下如何计算梯度。** 

在Graph模式时，我们在构建模型前向图时，同时也会构建梯度图，这样实际喂数据执行时可以很方便计算梯度。

但是eager执行是动态的，这就需要每一次执行都要记录这些操作以计算梯度，这是通过tf.GradientTape来追踪所执行的操作以计算梯度

```python
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
  loss = w * w + 3. * w + 8.
grad = tape.gradient(loss, w)
print(grad)

# tf.Tensor([[5.]], shape=(1, 1), dtype=float32)
```

对于eager执行，每个tape会记录当前所执行的操作，这个tape只对当前计算有效，并计算相应的梯度。

PyTorch也是动态图模式，但是与TensorFlow不同，它是每个需要计算Tensor会拥有```grad_fn```以追踪历史操作的梯度。


### 2.AutoGraph

TensorFlow 2.0引入的eager提高了代码的简洁性，而且更容易debug。

> 但是对于性能来说，**eager执行相比Graph模式会有一定的损失**。这不难理解，毕竟原生的Graph模式是先构建好静态图，然后才真正执行。这对于在分布式训练、性能优化和生产部署方面具有优势。
但是好在，**TensorFlow 2.0引入了tf.function和AutoGraph来缩小eager执行和Graph模式的性能差距，其核心是将一系列的Python语法转化为高性能的graph操作**。

AutoGraph在TensorFlow 1.x已经推出，```主要是可以将一些常用的Python代码转化为TensorFlow支持的Graph代码```。

一个典型的例子是在TensorFlow中我们必须使用```tf.while和tf.cond```等复杂的算子来实现动态流程控制。


In [26]:
'''
但是现在我们可以使用Python原生的for和if等语法写代码，然后采用AutoGraph转化为TensorFlow所支持的代码。
'''
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0.0
    return x

# eager 模式
print('Eager results: %2.2f, %2.2f' % (square_if_positive(tf.constant(9.0)),
                                       square_if_positive(tf.constant(-9.0))))
# graph 模式
tf_square_if_positive = tf.autograph.to_graph(square_if_positive)
with tf.Graph().as_default():
  # The result works like a regular op: takes tensors in, returns tensors.
  # You can inspect the graph using tf.get_default_graph().as_graph_def()
    g_out1 = tf_square_if_positive(tf.constant( 9.0))
    g_out2 = tf_square_if_positive(tf.constant(-9.0))
    with tf.compat.v1.Session() as sess:
        print('Graph results: %2.2f, %2.2f\n' % (sess.run(g_out1), sess.run(g_out2)))

# 上面我们定义了一个square_if_positive函数，它内部使用的Python的原生的if语法，对于TensorFlow 2.0的eager执行，这是没有问题的。
# 然而这是TensorFlow 1.x所不支持的，但是使用AutoGraph可以将这个函数转为Graph函数，你可以将其看成一个常规TensorFlow op，其可以在Graph模式下运行（tf2 没有Session，这是tf1.x的特性，想使用tf1.x的话需要调用tf.compat.v1）。
# 大家要注意eager模式和Graph模式的差异，尽管结果是一样的，但是Graph模式更高效。

Eager results: 81.00, 0.00
Graph results: 81.00, 0.00



In [27]:
'''
从本质上讲，AutoGraph是将Python代码转为TensorFlow原生的代码，我们可以进一步看到转化后的代码
'''
print(tf.autograph.to_code(square_if_positive))

# 可以看到AutoGraph转化的代码定义了两个条件函数，然后调用if_stmt op，应该就是类似tf.cond的op。

from __future__ import print_function

def tf__square_if_positive(x):
  try:
    with ag__.function_scope('square_if_positive'):
      do_return = False
      retval_ = None
      cond = ag__.gt(x, 0)

      def if_true():
        with ag__.function_scope('if_true'):
          x_1, = x,
          x_1 = x_1 * x_1
          return x_1

      def if_false():
        with ag__.function_scope('if_false'):
          x = 0.0
          return x
      x = ag__.if_stmt(cond, if_true, if_false)
      do_return = True
      retval_ = x
      return retval_
  except:
    ag__.rewrite_graph_construction_error(ag_source_map__)



tf__square_if_positive.autograph_info__ = {}



In [30]:
'''
AutoGraph支持很多Python特性，比如循环.
对于大部分Python特性AutoGraph是支持的，但是其仍然有限制，具体可以见Capabilities and Limitations。
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/LIMITATIONS.md
'''
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

print('Eager result: %d' % sum_even(tf.constant([10,12,15,20])))

tf_sum_even = tf.autograph.to_graph(sum_even)
with tf.Graph().as_default(), tf.compat.v1.Session() as sess:
    print('Graph result: %d\n' % sess.run(tf_sum_even(tf.constant([10,12,15,20]))))

Eager result: 42
Graph result: 42



In [35]:
'''
此外，要注意的一点是，经过AutoGraph转换的新函数是可以eager模式下执行的，但是性能却并不会比转换前的高.
'''
import timeit

x = tf.constant([10, 12, 15, 20])
print("Eager at orginal code:", timeit.timeit(lambda: sum_even(x), number=100))
print("Eager at autograph code:", timeit.timeit(lambda: tf_sum_even(x), number=100))
with tf.Graph().as_default(), tf.compat.v1.Session() as sess:
    graph_op = tf_sum_even(tf.constant([10, 12, 15, 20]))
    sess.run(graph_op)  # remove first call
    print("Graph at autograph code:", timeit.timeit(lambda: sess.run(graph_op), number=100))
    
# 从结果上看，Graph模式下的执行效率是最高的，原来的代码在eager模式下效率次之，经AutoGraph转换后的代码效率最低。

Eager at orginal code: 0.1542729390785098
Eager at autograph code: 0.3119836784899235
Graph at autograph code: 0.05139109306037426


### 3.tf.function性能优化

尽管eager执行更简洁，但是Graph模式却是性能更高，为了减少这个性能gap，TensorFlow 2.0引入了tf.function
> 在TensorFlow 2.0，我们一般不会直接使用 `tf.autograph`，因为eager执行下效率没有提升。要真正达到Graph模式下的效率，要依赖 ```tf.function``` 这个更强大的利器。

简单来说，就是tf.function可以将一个func中的TensorFlow操作构建为一个Graph，这样在调用时是执行这个Graph，这样计算性能更优.

- 3.1 [计算性能更优]()
- 3.2 [调用便捷]()
- 3.3 [多态性polymorphism]()
- 3.4 [指定输入参数类型 input_signature]()
- 3.5 [指定参数autograph]()
- 3.6 [应用到类方法中]()
- 3.7 [tf.print看具体数值]()
- 3.8 [dubug时设置tf.config.experimental_run_functions_eagerly=True]()

具体内容请看 [A02tf.function](./A02tf.function.ipynb)



### 4.tf.keras模型构建

在tf.layers以及tf.contrib.slim等高级API来创建模型，但是`2.0仅仅支持tf.keras.layers`，无论如何，省的大家重复造轮子，也意味着模型构建的部分大家都是统一的，增加代码的复用性（回忆一下原来的TensorFlow模型构建真是千奇百怪）。

值得注意的tf.nn模块依然存在，里面是各种常用的nn算子，不过大部分人不会去直接用这些算子构建模型，因为keras.layers基本上包含了常用的网络层。

- [4.1 采用Keras原有方式]()

如 采用`tf.keras.Sequential`

```python
model = tf.keras.Sequential([
layers.Dense(64, activation='relu', input_shape=(32,)),
layers.Dense(64, activation='relu'),
layers.Dense(10, activation='softmax')])
```

- [4.2 采用keras的functional API]()

```python
inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')
```

- [4.3 自定义layer&model]()

继承`tf.keras.layers.Layer`

继承`tf.keras.Model`,创建包含多layers的模块或者模型


具体内容请看 [A03模型构建](./A03model-build.ipynb)

### 5.模型训练

具体内容请看 [A04模型训练](A04training.ipynb), [A05分布式训练](A05distribute-training.ipynb)
